In [1]:
pip install opencv-python dlib pandas scipy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os

if not os.path.isfile("shape_predictor_68_face_landmarks.dat"):
    print("Error: Model file not found. Download it from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
    exit()


In [ ]:
import cv2
import dlib
import pandas as pd
import numpy as np
from scipy.spatial import distance
import os

# Load the models
dnn_model = "opencv_face_detector_uint8.pb"
dnn_proto = "opencv_face_detector.pbtxt"
net = cv2.dnn.readNetFromTensorflow(dnn_model, dnn_proto)
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# EAR and MAR calculation functions
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[2], mouth[6])
    B = distance.euclidean(mouth[3], mouth[5])
    C = distance.euclidean(mouth[0], mouth[4])
    return (A + B) / (2.0 * C)

def head_tilt_angle(nose, chin):
    dx = chin[0] - nose[0]
    dy = chin[1] - nose[1]
    return np.degrees(np.arctan2(dy, dx))

# Improved face detection function using DNN
def detect_faces_dnn(image):
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
    net.setInput(blob)
    detections = net.forward()

    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:   # Confidence threshold
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            faces.append((x, y, x1 - x, y1 - y))

    return faces

# Function to extract features from images
def extract_features(image_path, label):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None

    image = cv2.resize(image, (400, 400))  # Resize for consistency
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detect_faces_dnn(image)  # Use DNN for face detection

    features = []
    
    if len(faces) == 0:
        print(f"No face detected in {image_path}")
        return None

    for (x, y, w, h) in faces:
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        
        # Landmark prediction
        shape = predictor(gray, rect)
        shape = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])

        # Extract facial landmarks
        left_eye = shape[36:42]
        right_eye = shape[42:48]
        mouth = shape[60:68]
        nose = shape[27]
        chin = shape[8]

        ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0
        mar = mouth_aspect_ratio(mouth)
        tilt = head_tilt_angle(nose, chin)

        features.append([ear, mar, tilt, label])

    return features

# Directory containing images
base_dir = "train"
labels = ["awake_images", "drowsy_images"]
data = []

# Extract features from all images
for label in labels:
    label_path = os.path.join(base_dir, label)
    
    for img_name in os.listdir(label_path):
        img_path = os.path.join(label_path, img_name)
        
        try:
            features = extract_features(img_path, label)
            
            if features:
                data.extend(features)
        except Exception as e:
            print(f"Error processing {img_name}: {e}")

# Save features to CSV
df = pd.DataFrame(data, columns=["EAR", "MAR", "Tilt", "Label"])
df.to_csv("facial_features.csv", index=False)
print("Feature extraction complete and saved to facial_features.csv")


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
# ---------------------------
# 🚀 Load and preprocess the dataset
# ---------------------------
# Load the CSV
df = pd.read_csv("facial_features.csv")

# Extract features and labels
X = df[["EAR", "MAR", "Tilt"]].values
y = df["Label"].map({'awake_images': 0, 'drowsy_images': 1}).values  # Map labels

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape into 3D tensor -> (samples, height, width, channels)
X_reshaped = X.reshape(-1, 3, 1, 1)

# One-hot encode labels for CNN
y_cnn = to_categorical(y)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_cnn, test_size=0.2, random_state=42)

# ---------------------------
# ⚙️ Define CNN model
# ---------------------------
model = Sequential()

# Input layer
model.add(Conv2D(32, kernel_size=(2, 1), activation='relu', input_shape=(3, 1, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(0.3))

# Hidden layers
model.add(Conv2D(64, kernel_size=(2, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(0.3))

model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 classes: awake, drowsy

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# ---------------------------
# 🔥 Train the CNN
# ---------------------------
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

# ---------------------------
# 📊 Evaluate the model
# ---------------------------
loss, accuracy = model.evaluate(X_test, y_test)
print("\n✅ Test Accuracy:", accuracy)

# Classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred_classes))
print("\n🔎 Confusion Matrix:\n", confusion_matrix(y_true, y_pred_classes))


Epoch 1/100


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.5567 - loss: 0.7419 - val_accuracy: 0.5668 - val_loss: 0.6934
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5914 - loss: 0.7126 - val_accuracy: 0.5989 - val_loss: 0.6811
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6392 - loss: 0.6746 - val_accuracy: 0.6096 - val_loss: 0.6788
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6461 - loss: 0.6700 - val_accuracy: 0.6417 - val_loss: 0.6698
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6245 - loss: 0.6637 - val_accuracy: 0.6417 - val_loss: 0.6699
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6296 - loss: 0.6777 - val_accuracy: 0.6310 - val_loss: 0.6664
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6284 - loss: 0.6696 - val_accuracy: 0.6417 - val_loss: 0.6661
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6192 - loss: 0.6586 - val_accuracy: 0.6524 - val_lo

In [13]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\n✅ Test Accuracy:", loss)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6853 - loss: 0.6224 

✅ Test Accuracy: 0.6303648352622986
